In [8]:
import pandas
import numpy
import pymysql

conn = pymysql.connect(
    host='221.138.0.16',
    user='root',
    password='#dp5csv5serv',
    db='project_3',
    charset='utf8mb4',
    autocommit=True,
    cursorclass=pymysql.cursors.DictCursor
)

sql = 'Select * From no_show'

data = pandas.read_sql_query(sql, conn)
data

c:\Users\User\anaconda3\envs\alone\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,seq,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No_show
0,1,29872499824296,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,2,558997776694438,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,3,4262962299951,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,4,867951213174,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,5,8841186448183,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,110523,2572134369293,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,110524,3596266328735,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,110525,15576631729893,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,110526,92134931435557,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


In [9]:
# EDA
data.drop(columns=['seq','PatientId','AppointmentID','ScheduledDay','AppointmentDay','Neighbourhood'], inplace=True)
data.drop_duplicates()
data.dropna(axis=0)

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No_show
0,F,62,0,1,0,0,0,0,No
1,M,56,0,0,0,0,0,0,No
2,F,62,0,0,0,0,0,0,No
3,F,8,0,0,0,0,0,0,No
4,F,56,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...
110522,F,56,0,0,0,0,0,1,No
110523,F,51,0,0,0,0,0,1,No
110524,F,21,0,0,0,0,0,1,No
110525,F,38,0,0,0,0,0,1,No


In [10]:
data['Gender'] = data['Gender'].map({'F':0,'M':1})
data['No_show'] = data['No_show'].map({'No':0,'Yes':1})
data = data.astype(int)

In [13]:
target = 'No_show'

In [15]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, train_size=0.8, test_size=0.2, stratify=data[target])
X_train = data.drop(columns=[target])
y_train = data[target]
X_val = data.drop(columns=[target])
y_val = data[target]

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    SimpleImputer(), 
    DecisionTreeClassifier(random_state=1, criterion='entropy')
)
pipe.fit(X_train, y_train)
pipe.score(X_val, y_val)

0.8018493218851502

In [17]:
temp = [1,50,1,0,1,0,0,1]
temp1 = [0,47,0,1,1,1,1,1]
y_predict = pipe.predict([temp1])
y_predict

c:\Users\User\anaconda3\envs\alone\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([1])

In [18]:
import pickle

with open('model.pkl','wb') as pickle_file:
    pickle.dump(pipe, pickle_file)